In [ ]:
import torch

# 현재 사용 중인 GPU의 속성 정보 가져오기
gpu_stats = torch.cuda.get_device_properties(0)  # 첫 번째 GPU 속성 정보를 가져옴

# 현재 GPU에서 예약된 메모리 양을 GB 단위로 계산하여 출력
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
print(f"{start_gpu_memory} GB of memory reserved.")  # 예약된 메모리 양 출력

# GPU의 전체 메모리 크기를 GB 단위로 계산하여 출력
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")  # GPU 이름과 최대 메모리 출력

In [ ]:
import torch
import gc

# GPU 메모리 초기화
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from unsloth import FastLanguageModel
import torch

# 모델의 데이터 타입을 설정 (bfloat16: 16비트 정밀도를 사용하여 메모리 사용량 절감)
dtype = torch.bfloat16

# 모델을 4비트 양자화하여 로드할지 여부 (메모리 절약을 위해 True로 설정)
load_in_4bit = True

# 사전 학습된 모델과 토크나이저 불러오기
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Bllossom/llama-3.2-Korean-Bllossom-3B",  # 사전 학습된 모델의 이름
    max_seq_length=200,  # 입력 시퀀스의 최대 길이 제한 설정
    dtype=dtype,  # 위에서 설정한 데이터 타입을 적용
    load_in_4bit=load_in_4bit,  # 4비트 양자화를 적용하여 메모리 최적화
    device_map="auto", # 모델을 자동으로 GPU 또는 CPU에 할당하도록 설정
)

tokenizer.padding_side = "right"  # 오른쪽 패딩

# 체크포인트 경로
checkpoint_path = "./outputs/checkpoint-3180"

In [ ]:
from datasets import load_dataset, concatenate_datasets

kmmlu_categories = [
    "Accounting", "Agricultural-Sciences", "Aviation-Engineering-and-Maintenance", "Biology", 
    "Chemical-Engineering", "Chemistry", "Civil-Engineering", "Computer-Science", "Construction", 
    "Criminal-Law", "Ecology", "Economics", "Education", "Electrical-Engineering", 
    "Electronics-Engineering", "Energy-Management", "Environmental-Science", "Fashion", "Food-Processing", 
    "Gas-Technology-and-Engineering", "Geomatics", "Health", "Industrial-Engineer", "Information-Technology", "Interior-Architecture-and-Design", 
    "Law", "Machine-Design-and-Manufacturing", "Management", "Maritime-Engineering", "Marketing", "Materials-Engineering",
    "Mechanical-Engineering", "Nondestructive-Testing", "Patent", "Political-Science-and-Sociology", "Psychology",
    "Public-Safety", "Railway-and-Automotive-Engineering", "Real-Estate", "Refrigerating-Machinery", "Social-Welfare",
    "Taxation", "Telecommunications-and-Wireless-Technology", "Korean-History", "Math"
]

In [ ]:
# 모든 카테고리 저장할 리스트
eval_list = []
train_list = []

for category in kmmlu_categories:
    # KMMLU 카테고리별 데이터셋 불러오기
    dataset = load_dataset("HAERAE-HUB/KMMLU", category, num_proc=8)
    # train과 dev 데이터셋 각각 저장
    train_dataset = dataset['train']
    eval_dataset = dataset['dev']
    
    train_list.append(train_dataset)
    eval_list.append(eval_dataset)

In [ ]:
# 모든 데이터셋을 하나로 합치기
train_dataset = concatenate_datasets(train_list)
eval_dataset = concatenate_datasets(eval_list)

In [ ]:
print(f"학습 데이터: {len(train_dataset)}개")
print(f"평가 데이터: {len(eval_dataset)}개")

In [ ]:
prompt_template = """
문제: {question}
A. {A}
B. {B}
C. {C}
D. {D}
정답:{answer}"""

EOS_TOKEN = tokenizer.eos_token

In [ ]:
def preprocess_function(texts):
    question = texts['question']
    A = texts['A']
    B = texts['B']
    C = texts['C']
    D = texts['D']
    answer = []
    for idx in texts['answer']:
        answer.append(['A', 'B', 'C', 'D'][idx-1])

    result = []
    
    for q, a, b, c, d, ans in zip(question, A, B, C, D, answer):
        text = prompt_template.format(question=q, A=a, B=b, C=c, D=d, answer=ans)
        text += EOS_TOKEN
        result.append(text)
        
    return {
        "text": result
    }

In [ ]:
# 전처리 적용
train_dataset = train_dataset.map(
    preprocess_function, 
    batched=True, 
    num_proc=4,
    remove_columns=train_dataset.column_names
)

eval_dataset = eval_dataset.map(
    preprocess_function, 
    batched=True, 
    num_proc=4,
    remove_columns=eval_dataset.column_names
)

# 총 스텝 수 계산
total_steps = len(train_dataset) // (4 * 4)  # batch_size * gradient_accumulation_steps

In [ ]:
# 모델 초기화
model = FastLanguageModel.get_peft_model(
    model,
    r=8,  # 0보다 큰 어떤 숫자도 선택 가능! 8, 16, 32, 64, 128이 권장됩니다.
    lora_alpha=16,  # LoRA 알파 값을 설정합니다. # 튜토리얼은 16
    lora_dropout=0.01,  # 드롭아웃을 지원합니다. # # Supports any, but = 0 is optimized
    target_modules=[
        "q_proj", # query
        "k_proj", # key
        "v_proj", # value
        "o_proj", # output (어텐션 최종 출력)
        "gate_proj", # 게이트 조절 시 사용
        "up_proj", # 차원 확장 시 사용
        "down_proj", # 차원 축소 시 사용
    ],  # 타겟 모듈을 지정합니다.
    bias="none",  # 바이어스를 지원합니다.
    # True 또는 "unsloth"를 사용하여 매우 긴 컨텍스트에 대해 VRAM을 30% 덜 사용하고, 2배 더 큰 배치 크기를 지원합니다.
    use_gradient_checkpointing="unsloth",
    random_state=3407,  # 난수 상태를 설정합니다. # 공식 : 3407
    use_rslora=True,  # 순위 안정화 LoRA를 지원합니다.
    loftq_config=None,  # LoftQ를 지원합니다.
)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

training_args = TrainingArguments(
    # 배치 설정
    per_device_train_batch_size=4,       # 훈련 시 배치 사이즈
    per_device_eval_batch_size=4,        # 평가 시 배치 사이즈
    gradient_accumulation_steps=4,       # gradient 누적 단계 수 (메모리 최적화)

    # 웜업 및 에폭
    warmup_ratio=0.1,                    # 전체 스텝 대비 warmup 비율
    num_train_epochs=1,                  # 총 학습 에폭 수

    # 최적화 및 학습률 스케줄링
    learning_rate=2e-4,                  # 초기 학습률
    optim="adamw_bnb_8bit",              # 8비트 AdamW 최적화
    weight_decay=0.01,                   # 가중치 감소율
    lr_scheduler_type="cosine",          # 코사인 학습률 스케줄러

    # 하드웨어 호환이 필요한 최적화
    fp16=not torch.cuda.is_bf16_supported(),  # FP16 사용 (BF16 미지원 시)
    bf16=torch.cuda.is_bf16_supported(),      # BF16 사용 (지원 시)
    torch_compile=True,                       # PyTorch 컴파일을 통한 메모리, 학습 최적화

    # 로깅 및 wandb
    logging_steps=total_steps // 20,       # 전체 스텝의 5%마다 로깅
    report_to="wandb",                    # wandb
    run_name="KMMLU_ALL_CATEGORIES",      # run 이름

    # 평가 및 체크포인트
    eval_strategy="steps",                # 정기적인 평가
    eval_steps=total_steps // 20,         # 전체 스텝의 5%마다 평가
    load_best_model_at_end=True,          # 최고 성능 모델 저장
    metric_for_best_model="eval_loss",         # 성능 평가 기준 (loss)
    greater_is_better=False,              # 낮은 loss가 더 좋은 모델로 평가

    # 체크포인트 저장
    save_strategy="steps",                # 체크포인트 저장 전략 (스텝 단위)
    save_steps=total_steps // 20,         # 전체 스텝의 5%마다 저장
    save_total_limit=5,                   # 최대 체크포인트 저장 개수

    # 기타 설정
    seed=3407,                            # 시드 고정
    output_dir="outputs",                 # 출력 경로
    max_grad_norm=1.0,                    # 최대 gradient norm 설정
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field="text",
    dataset_num_proc=4,
    packing=False,
    args=training_args,
)

In [ ]:
import wandb
from huggingface_hub import HfApi
from peft import PeftModel
import os
import json

wandb.login()

# WandB 설정
wandb.init(
    project="kmmlu-finetuning",  # 프로젝트명 
    config={
        "model_name": "Bllossom/llama-3.2-Korean-Bllossom-3B",
        "dataset": "HAERAE-HUB/KMMLU",
        "categories": kmmlu_categories,  # 사용된 카테고리 목록
        "train_samples": len(train_dataset),
        "eval_samples": len(eval_dataset),
        "batch_size": training_args.per_device_train_batch_size,
        "learning_rate": training_args.learning_rate,
        "epochs": training_args.num_train_epochs,
        "warmup_ratio": training_args.warmup_ratio,
        "weight_decay": training_args.weight_decay,
        "lora_r": 8,  # LoRA rank
        "lora_alpha": 16,  # LoRA alpha
        "lora_dropout": 0.01,
        "max_seq_length": 200,
    },
    tags=["kmmlu", "llama", "lora"]
)

trainer_stats = trainer.train(resume_from_checkpoint=checkpoint_path)

# 학습 완료 후 모델과 어댑터 결합 및 업로드
def merge_and_upload_model(base_model, adapter_path, repo_id):
    """
    학습된 어댑터를 베이스 모델과 결합하고 HuggingFace Hub에 업로드
    
    Args:
        base_model: 기본 모델
        adapter_path: 학습된 어댑터 경로
        repo_id: 업로드할 HuggingFace 레포지토리 ID (예: "username/model-name")
    """
    # 임시 저장 경로
    merged_model_path = "./merged_model"
    
    # 모델과 어댑터 결합
    print("Merging model with adapter...")
    merged_model = base_model.merge_and_unload()
    
    # 결합된 모델 저장
    print(f"Saving merged model to {merged_model_path}...")
    merged_model.save_pretrained(merged_model_path, safe_serialization=True)
    tokenizer.save_pretrained(merged_model_path)
    
    # config 파일 저장
    config_path = os.path.join(merged_model_path, "config.json")
    with open(config_path, "w", encoding="utf-8") as f:
        json.dump({
            "base_model": "Bllossom/llama-3.2-Korean-Bllossom-3B",
            "training_dataset": "HAERAE-HUB/KMMLU",
            "categories": kmmlu_categories,
            "max_seq_length": 200,
            "prompt_template": prompt_template,
        }, f, indent=2, ensure_ascii=False)
    
    # HuggingFace Hub에 업로드
    print(f"Uploading merged model to {repo_id}...")
    api = HfApi()
    api.create_repo(repo_id, exist_ok=True)
    api.upload_folder(
        folder_path=merged_model_path,
        repo_id=repo_id,
        commit_message="Upload merged model with KMMLU training"
    )
    
    print("Model successfully uploaded to HuggingFace Hub!")
        
    finally:
        # 임시 파일 정리
        if os.path.exists(merged_model_path):
            import shutil
            shutil.rmtree(merged_model_path)
            
# 학습 완료 후 사용 예시:
repo_id = "Yeongi/llama-kmmlu-PEFT-finetuned"  # 본인의 허깅페이스 레포지토리 ID로 변경
merge_and_upload_model(model, checkpoint_path, repo_id)

wandb.finish()

In [ ]:
# GPU 메모리 초기화
gc.collect()
torch.cuda.empty_cache()